In [17]:
#!pip install -q pyomo
#!apt-get install -y -qq coinor-cbc

In [18]:
from pyomo.environ import *
import numpy as np

In [19]:
coefs = np.loadtxt('sample_data/lab4_ex1_coef.txt',delimiter=',')

In [20]:
coefs

array([[  0.85,   3.75,  -8.75,  -3.45, -22.38,  20.  ,   0.  ],
       [ -0.7 ,  -3.9 ,  -1.  ,   0.  ,   1.  ,   0.  ,  14.9 ],
       [  1.  ,   0.  ,   0.  ,  -1.  ,   0.  ,  -1.  ,   0.9 ],
       [  0.  ,  -3.  ,  -2.  ,   0.  ,   0.  ,  -6.  , -51.7 ],
       [  0.  ,   0.  ,   0.  ,   4.8 ,   9.12,   7.2 ,  75.  ],
       [ -0.8 ,  -1.5 ,   8.  ,  -2.  ,   1.  ,   0.  , -27.  ]])

In [21]:
consrhs = coefs[1:,-1]
concoefs = coefs[1:,:-1]
n=6
m = 5

objcoef = coefs[0,:-1]
cols = np.arange(n)
rows=np.arange(m)

In [22]:
m1 = ConcreteModel()
m1.cons = ConstraintList()
m1.x = Var(cols)

for i in rows:
    m1.cons.add(summation(concoefs[i],m1.x)<=consrhs[i])

for j in cols:
    m1.x[j].setlb(0)
    m1.x[j].setub(np.inf)

In [23]:
m1.cost = Objective(expr = summation(objcoef,m1.x),sense=maximize)

In [24]:
solver = SolverFactory('cbc')


In [25]:
opt = solver.solve(m1)
#opt.write()

    model.name="unknown";
      - termination condition: unbounded
      - message from solver: <undefined>


### 7)

In [26]:
print('Termination condition:',opt.solver.termination_condition)
print('Solver status message:',opt.solver.status)

Termination condition: unbounded
Solver status message: warning


Termination condition unbounded implies that the objective function has no upper bound.
Status message warning implies the problem was not solved.

### 9)

The objective function has no upper bound (subject to given constraints), but it may have a lower bound. We can check this by changing it to a minimization problem.

In [27]:
m1a = m1.clone()
m1a.cost.set_sense(minimize)

In [28]:
opt1a = solver.solve(m1a)
print('Termination condition:',opt1a.solver.termination_condition)
print('Solver status message:',opt1a.solver.status)

Termination condition: optimal
Solver status message: ok


In [29]:
m1a.cons.display()

cons : Size=5
    Key : Lower : Body                : Upper
      1 :  None :         -82.1158942 :  14.9
      2 :  None :                 0.9 :   0.9
      3 :  None :          -69.007368 : -51.7
      4 :  None :   74.99999990399999 :  75.0
      5 :  None : -26.999999799999998 : -27.0


In [30]:
print('The optimal value (minimum value) of the objective function is',m1a.cost())

The optimal value (minimum value) of the objective function is -97.02184239599997


In [31]:
for i in cols:    
    print('The value of variable x%d is'%(i+1),m1a.x[i]())

The value of variable x1 is 0.9
The value of variable x2 is 23.002456
The value of variable x3 is 0.0
The value of variable x4 is 0.0
The value of variable x5 is 8.2236842
The value of variable x6 is 0.0


Constraint 2, Constraint 4 and Constraint 5 are active constraints. 

Alternatively,if the problem has to be maximised, i.e there's no use of finding the min value of the objective,
 we can find the variables which are the reason for the unboundedness, and  put separate upper bounds for these variables and find the value of the cost at an arbitrary upper bound, as per our convenience (by this method we'll also get exact values (feasible) of the other variables, which we don't otherwise get from the solver in case of unbounded problems.

On inspection, we see that increasing the value of x2 doesn't increase the value of any of the constraints. For all other variables, at least one constraint value is increased. Hence, we can say that x2 can be made as large as possible without violating any constraint. And since x2 has a positive coefficient in the objective function, we can say that x2 is causing the unboundedness. So we can try to solve the problem by putting an upper bound on x2, until we get the objective function as large as we want